# Detect Document Types

This notebook queries the manifest table (`md_file_history`) to detect which document types
were extracted from the current processing run. It sets task values that downstream
condition tasks use to determine which pipelines to run.

**Output Task Values:**
- `has_tsdta`: "true" if tsDTA Excel files exist
- `has_protocol`: "true" if Protocol PDF files exist
- `has_operational_agreement`: "true" if Operational Agreement files exist

**Used by:** `job_cdm_dta_import` when `pipeline=auto`


In [ ]:
# Import required libraries
from pyspark.sql import functions as F
import json

print("✅ Libraries loaded")


In [ ]:
# Get parameters
# catalog_override is the primary way to specify the catalog
catalog_override = dbutils.widgets.get("catalog_override")

if not catalog_override or not catalog_override.strip():
    raise ValueError("catalog_override parameter is required")

catalog = catalog_override.strip()
bronze_schema = "bronze_md"

print(f"📦 Catalog: {catalog}")
print(f"📦 Bronze Schema: {bronze_schema}")


In [ ]:
# ============================================================================
# Query manifest table to detect document types
# We look at document_tags array and status to find ready-to-process documents
# ============================================================================

manifest_table = f"{catalog}.{bronze_schema}.md_file_history"

print(f"📋 Querying manifest table: {manifest_table}")

# Query documents that are ready for processing
# Status = 'READY_FOR_PROCESSING' indicates files just extracted
manifest_df = spark.table(manifest_table).filter(
    F.col("status") == "READY_FOR_PROCESSING"
)

total_docs = manifest_df.count()
print(f"📄 Found {total_docs} documents ready for processing")

if total_docs == 0:
    print("⚠️ No documents to process")
    # Set all flags to false
    dbutils.jobs.taskValues.set(key="has_tsdta", value="false")
    dbutils.jobs.taskValues.set(key="has_protocol", value="false")
    dbutils.jobs.taskValues.set(key="has_operational_agreement", value="false")
    dbutils.notebook.exit(json.dumps({"status": "success", "message": "No documents to process"}))


In [ ]:
# ============================================================================
# Analyze document types from document_tags array
# Document tags are set by the file processor based on folder structure/extension
# ============================================================================

# Explode document_tags array and count distinct values
tags_df = manifest_df.select(
    F.explode_outer(F.col("document_tags")).alias("tag")
).groupBy("tag").count()

print("\n📊 Document Tags Distribution:")
tags_df.show(truncate=False)

# Collect tags into a set for easy lookup
tags_list = [row.tag for row in tags_df.collect() if row.tag]
tags_set = set([t.upper() for t in tags_list if t])

print(f"\n📋 Detected tags: {tags_set}")


In [ ]:
# ============================================================================
# Determine which pipelines to run based on detected tags
# ============================================================================

# Check for tsDTA documents
# tsDTA files have tags like ["DTA", "tsDTA"] or just ["tsDTA"]
has_tsdta = "TSDTA" in tags_set

# Check for Protocol documents  
# Protocol files have tags like ["Protocol"] or ["DTA", "Protocol"]
has_protocol = "PROTOCOL" in tags_set

# Check for Operational Agreement documents
# OA files have tags like ["DTA", "OPERATIONAL_AGREEMENT"]
has_operational_agreement = "OPERATIONAL_AGREEMENT" in tags_set

print("\n✅ Pipeline Detection Results:")
print(f"  - has_tsdta: {has_tsdta}")
print(f"  - has_protocol: {has_protocol}")
print(f"  - has_operational_agreement: {has_operational_agreement}")


In [ ]:
# ============================================================================
# Set task values for downstream condition tasks
# ============================================================================

dbutils.jobs.taskValues.set(key="has_tsdta", value=str(has_tsdta).lower())
dbutils.jobs.taskValues.set(key="has_protocol", value=str(has_protocol).lower())
dbutils.jobs.taskValues.set(key="has_operational_agreement", value=str(has_operational_agreement).lower())

print("\n✅ Task values set successfully")
print(f"  - has_tsdta = {str(has_tsdta).lower()}")
print(f"  - has_protocol = {str(has_protocol).lower()}")
print(f"  - has_operational_agreement = {str(has_operational_agreement).lower()}")

dbutils.notebook.exit(json.dumps({
    "status": "success",
    "has_tsdta": has_tsdta,
    "has_protocol": has_protocol,
    "has_operational_agreement": has_operational_agreement
}))
